# VGG网络

In [2]:
import time
import torch
from torch import nn,optim

In [3]:
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def vgg_block(num_convs,in_channels,out_channels):
    blk = []
    for i in range(num_convs):
        if i== 0:
            blk.append(nn.Conv2d(in_channels,out_channels,kernel_size = 3,padding = 1))
        else:
            blk.append(nn.Conv2d(out_channels,out_channels,kernel_size = 3,padding = 1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size = 2,stride = 2))#使得高宽减半
    return nn.Sequential(*blk)

In [6]:
conv_arch = ((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
#经过5个vgg_block。高宽会减半5次，224/32 = 7
fc_features = 512*7*7
fc_hidden_units = 4096

In [12]:
def vgg(conv_arch,fc_features,fc_hidden_units = 4096):
    net = nn.Sequential()
    #卷积层部分
    for i,(num_convs,in_channels,out_channels) in enumerate(conv_arch):
        #每经过一个vgg_block都会使高宽减半
        net.add_module("vgg_block_" + str(i+1),
                      vgg_block(num_convs,in_channels,out_channels))
        #全连接层部分
    net.add_module("fc",nn.Sequential(d2l.FlattenLayer(),
                                         nn.Linear(fc_features,fc_hidden_units),
                                         nn.ReLU(),
                                         nn.Dropout(0.5),
                                         nn.Linear(fc_hidden_units,fc_hidden_units),
                                         nn.ReLU(),
                                         nn.Dropout(0.5),
                                         nn.Linear(fc_hidden_units,10)))
    return net

构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状

In [13]:
net = vgg(conv_arch,fc_features,fc_hidden_units)
X = torch.rand(1,1,224,224)

#named_children获取一级子模块及其名字(named_module会返回所有子模块，
#包括子模块的子模快)
for name,blk in net.named_children():
    X = blk(X)
    print(name,"output shape",X.shape)

vgg_block_1 output shape torch.Size([1, 64, 112, 112])
vgg_block_2 output shape torch.Size([1, 128, 56, 56])
vgg_block_3 output shape torch.Size([1, 256, 28, 28])
vgg_block_4 output shape torch.Size([1, 512, 14, 14])
vgg_block_5 output shape torch.Size([1, 512, 7, 7])
fc output shape torch.Size([1, 10])


可以看到，每次我们将输入的高和宽减半,直到最终高和宽变成7后传入连接层。
与此同时，输出通道数每次翻倍，直到变成512.因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高宽减半以及通道翻倍的设计，使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

## 获取数据和训练模型

因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练

In [14]:
ratio = 8
small_conv_arch = [(1,1,64//ratio),(1,64//ratio,128//ratio),
                  (2,128//ratio,256//ratio),
                  (2,256//ratio,512//ratio),
                  (2,512//ratio,512//ratio)]
net = vgg(small_conv_arch,fc_features//ratio,fc_hidden_units//ratio)
print(net)

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 模型训练过程

In [ ]:
batch_size = 64
#如出现”out of memory“的报错信息，可减小batch_size或resize
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size,resize = 224)
lr,num_epochs = 0.001,5
optimizer = torch.optim.Adam(net.parameters(),lr = lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cpu


### 小结

Vgg-11通过5个可以重复使用的卷积模块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型